In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('df_english.csv')

In [3]:
df.head(1)

,Unnamed: 0,Id,Price,Address,Nearest_Subway,Publication_date,Num_of_rooms,Total_area,Living_space,Floor,...,Cargo_lift,New_building_name,Building,Official_developer,Participation_type,Due_date,Link,Coordinates,WMO,Area
0,0,105724302,9800000,"Санкт-Петербург, Пушкинский р-н, пос. Шушары, ...",NaN,17.10.2022 в 10:16,3,73 м²,NaN,1 из 5,...,NaN,NaN,NaN,NaN,NaN,NaN,https://avito.ru//sankt-peterburg/kvartiry/3-k...,"59.737926, 30.461476",Шушары,Пушкинский


In [4]:
df.columns

Index(['Unnamed: 0', 'Id', 'Price', 'Address', 'Nearest_Subway',
       'Publication_date', 'Num_of_rooms', 'Total_area', 'Living_space',
       'Floor', 'Balcony', 'Room_type', 'Ceiling_height', 'Bathroom',
       'Windows', 'Repair', 'Furniture', 'Warm_floor', 'Decoration',
       'Appliances', 'Sale_method', 'Deal_type', 'House_type', 'Year_build',
       'Floors_in_house', 'Passenger_lift', 'Parking', 'In_house', 'Yard',
       'Cargo_lift', 'New_building_name', 'Building', 'Official_developer',
       'Participation_type', 'Due_date', 'Link', 'Coordinates', 'WMO', 'Area'],
      dtype='object')

In [5]:
# Избавимся от признаков, которые не несут информативности для нас.
df = df.drop(['Unnamed: 0', 'Id', 'Official_developer', 
              'New_building_name', 'Building', 'Link', 'Floors_in_house'], axis=1)

# Избавляем от признаков, у которых пропусков более 65%
df = df.dropna(thresh=int(len(df) * .65), axis=1)

# Избавляемся от дубликатов
df = df.drop_duplicates(['Address', 'Total_area', 'Floor'])

После того как мы удалили все ненужные признаки, у нас остались следующие фичи:

In [6]:
df.columns

Index(['Price', 'Address', 'Nearest_Subway', 'Publication_date',
       'Num_of_rooms', 'Total_area', 'Living_space', 'Floor', 'Balcony',
       'Bathroom', 'Windows', 'Sale_method', 'House_type', 'Coordinates',
       'WMO', 'Area'],
      dtype='object')

С ними мы и будем работать.

# Feature_engineering

### Обработка площади

In [7]:
# убираем м.кв. из "Total_area" и "Living_space"
def convert_area(value):
    correct_list = [str(i) for i in range(10)] + ['.']
    result = ''
    if pd.isna(value):
        return value
    for literal in str(value):
        if literal in correct_list:
            result += literal
    return result

for col in ["Total_area", "Living_space"]:
    df.loc[:, col] = df[col].apply(convert_area).astype(np.float32)

### Обработка координат

Ориентироваться на адрес не очень удобно, поэтому этот признак преобразуем в кооридинаты

In [8]:
df[['lat_object', 'lon_object']] = df['Coordinates'].str.split(', ',expand=True).astype('float')

df[['lat_object', 'lon_object']].describe()

,lat_object,lon_object
count,19565.000000,19565.000000
mean,59.675410,30.485540
std,2.044360,3.272827
min,38.552623,-4.719892
25%,59.852538,30.251297
50%,59.931927,30.326360
75%,60.000654,30.406679
max,60.262019,137.986851


Видно, что в данных есть выбросы, не характерные для координат СПб. Ограничив последние диапазоном по широте от 58,7 до 61 град и по долготе от 28,5 до 33,5, введем фильтр.

In [9]:
df[(df['lat_object'] > 61) | (df['lat_object'] < 58.7) | 
   (df['lon_object'] > 33.5) | (df['lon_object'] < 28.5)]['Price'].count()

348

In [10]:
df = df.drop(df[(df['lat_object'] > 61) | (df['lat_object'] < 58.7) | 
   (df['lon_object'] > 33.5) | (df['lon_object'] < 28.5)].index)

In [11]:
df.head()

,Price,Address,Nearest_Subway,Publication_date,Num_of_rooms,Total_area,Living_space,Floor,Balcony,Bathroom,Windows,Sale_method,House_type,Coordinates,WMO,Area,lat_object,lon_object
0,9800000,"Санкт-Петербург, Пушкинский р-н, пос. Шушары, ...",NaN,17.10.2022 в 10:16,3,73.0,NaN,1 из 5,балкон,раздельный,"во двор, на улицу, на солнечную сторону",свободная,кирпичный,"59.737926, 30.461476",Шушары,Пушкинский,59.737926,30.461476
1,24000000,"Санкт-Петербург, наб. реки Фонтанки, 28",Гостиный двор11–15 мин.Невский проспект11–15 м...,17.10.2022 в 01:22,3,85.0,NaN,3 из 5,NaN,совмещенный,"во двор, на солнечную сторону",свободная,кирпичный,"59.939086, 30.343213",Литейный округ,Центральный,59.939086,30.343213
2,12500000,"Санкт-Петербург, Камышовая ул., 3к1",Комендантский проспект6–10 мин.Старая деревняо...,22.10.2022 в 10:29,2,55.0,NaN,8 из 12,лоджия,раздельный,на улицу,альтернативная,панельный,"60.005504, 30.254046",округ Озеро Долгое,Приморский,60.005504,30.254046
3,14200000,"Санкт-Петербург, ул. Беринга, 1",Приморскаяот 31 мин.Василеостровскаяот 31 мин.,21.10.2022 в 16:53,1,50.0,32.0,17 из 17,балкон,совмещенный,NaN,альтернативная,монолитный,"59.935831, 30.249087",округ Гавань,Василеостровский,59.935831,30.249087
4,5500000,"Санкт-Петербург, Пулковская ул., 8к2",Звёздная6–10 мин.Купчино21–30 мин.Московская21...,16.10.2022 в 03:52,студия,26.0,14.0,9 из 25,балкон,совмещенный,"во двор, на солнечную сторону",свободная,монолитный,"59.838028, 30.352169",округ Звёздное,Московский,59.838028,30.352169


In [12]:
#удаляем лишний столбец 
df = df.drop(["Coordinates"], axis=1)
# может еще понадобится удалить столбец Address

С КООРДИНАТАМИ НУЖНО ЕЩЕ ЗАМОРОЧИТЬСЯ И ИЗМЕРИТЬ РАССТОЯНИЕ ДО ЦЕНТРА!!!

### Обработка "Floor"

In [13]:
# разделим общее количество этажей в доме
# и квариру на конкретном этаже.
new_df = df['Floor'].str.split('из',expand=True)

#переименуем наши фичи
new_df.columns=['Flat_floor','Total_floor']

# объединим два датафрейма
df = pd.concat([df,new_df],axis=1)

# удалим ненужную фичу
df = df.drop(['Floor'], axis=1)

In [14]:
df.head()

,Price,Address,Nearest_Subway,Publication_date,Num_of_rooms,Total_area,Living_space,Balcony,Bathroom,Windows,Sale_method,House_type,WMO,Area,lat_object,lon_object,Flat_floor,Total_floor
0,9800000,"Санкт-Петербург, Пушкинский р-н, пос. Шушары, ...",NaN,17.10.2022 в 10:16,3,73.0,NaN,балкон,раздельный,"во двор, на улицу, на солнечную сторону",свободная,кирпичный,Шушары,Пушкинский,59.737926,30.461476,1,5
1,24000000,"Санкт-Петербург, наб. реки Фонтанки, 28",Гостиный двор11–15 мин.Невский проспект11–15 м...,17.10.2022 в 01:22,3,85.0,NaN,NaN,совмещенный,"во двор, на солнечную сторону",свободная,кирпичный,Литейный округ,Центральный,59.939086,30.343213,3,5
2,12500000,"Санкт-Петербург, Камышовая ул., 3к1",Комендантский проспект6–10 мин.Старая деревняо...,22.10.2022 в 10:29,2,55.0,NaN,лоджия,раздельный,на улицу,альтернативная,панельный,округ Озеро Долгое,Приморский,60.005504,30.254046,8,12
3,14200000,"Санкт-Петербург, ул. Беринга, 1",Приморскаяот 31 мин.Василеостровскаяот 31 мин.,21.10.2022 в 16:53,1,50.0,32.0,балкон,совмещенный,NaN,альтернативная,монолитный,округ Гавань,Василеостровский,59.935831,30.249087,17,17
4,5500000,"Санкт-Петербург, Пулковская ул., 8к2",Звёздная6–10 мин.Купчино21–30 мин.Московская21...,16.10.2022 в 03:52,студия,26.0,14.0,балкон,совмещенный,"во двор, на солнечную сторону",свободная,монолитный,округ Звёздное,Московский,59.838028,30.352169,9,25


### Обработка Num_of_rooms

In [15]:
df.loc[df['Num_of_rooms'] == 'студия', ['Num_of_rooms']] = 0
df.loc[df['Num_of_rooms'] == '10 и больше', ['Num_of_rooms']] = 10
df = df.loc[df['Num_of_rooms'] != 'свободная планировка']

In [16]:
df['Num_of_rooms'].unique()

array(['3', '2', '1', 0, '4', '5', '7', '6', nan, 10, '8', '9'],
      dtype=object)

# Что делать со свободной планировкой?

### Вытаскиваем уникальные значения из некоторых категориальных фичей

In [17]:
# #это метод OHE.
# def convert_lists(df: pd.DataFrame) -> pd.DataFrame:
#     cols = [
#         "Balcony", "Bathroom", "Windows", "Sale_method",
#         "House_type"
#            ]
#     new_cols = []
#     for col in cols:
#         series = df.loc[:, col]
#         data = []
#         unique_values = set()
#         new_cols_data = []

#         for row in series.values:
#             row_c = [value.strip() for value in (row.split(",") if isinstance(row, str) else [])]
#             for element in row_c:
#                 if not element in unique_values:
#                     unique_values.add(element)
#             data.append(row_c)
    
#         unique_values = list(unique_values)

#         for row in data:
#             new_cols_data.append([1 if value in row else 0 for value in unique_values])
    
#         col_names = [f"{col}_{i+1}" for i in range(len(unique_values))]
#         new_cols.append(pd.DataFrame(data=new_cols_data, columns=col_names, dtype=np.int8))
#         print("\n".join(map(str,zip(unique_values, col_names))))

#         df.drop(columns=[col], axis=1, inplace=True)

#     return pd.concat([df]+new_cols, axis=1)

# df = convert_lists(df)